# Lecture 3: Spark Streaming

Spark streaming is an extension of the Spark API that enables scalabe stream processing. The stream of data can be ingested from many data sources such as *Kafka*, *s3* or *TCP socket*. The Spark API allows to process data via high-level functions such as *map* and *reduce*. As we are going to se, it is also possible to use dataframe operations. 

Processed data can be exported to an external database and used to make live dashboards or offline analysis. 

Spark streaming works by dividing the input data into micro-batches that can be treated as static dataset. In Spark this is called *discretized stream* (*DStream*). The DStream is represented using RDDs.

![DStream](imgs/lecture3/DStream.png)

A transformation of the DStream, i.e. `Dstream.map` will act independently on each batch. For example, in the image below, we can filter the original RDD to remove some data and produce a new stream. 

![DStream_filter](imgs/lecture3/DStream_filter.png)

In this lecture we will see how to setup a simple stream using as source a socket.

## Setup

In [ ]:
# set this variable with one of the following values
# -> 'local'
# -> 'docker_container'
# -> 'docker_cluster'
CLUSTER_TYPE ='docker_container'

In [ ]:
# set env variable
%env CLUSTER_TYPE $CLUSTER_TYPE

## Start the cluster 

Environment variables need to be set only in the case of a local cluster

In [ ]:
if CLUSTER_TYPE=='local':
    import findspark
    findspark.init('/Users/matteo/Work/MAPD/spark-3.1.1-bin-hadoop3.2/')

In [ ]:
%%script bash --no-raise-error

if [[ "$CLUSTER_TYPE" != "docker_cluster" ]]; then
    echo "Launching master and worker"
    
    # start master 
    $SPARK_HOME/sbin/start-master.sh --host localhost \
        --port 7077 --webui-port 8080
    
    # start worker
    $SPARK_HOME/sbin/start-worker.sh spark://localhost:7077 \
        --cores 4 --memory 2g
fi

## Create the Spark session

Later on we will explain what is the role of [Apache Arrow](), but first we need to install it and create the spark session with it.

In [ ]:
from pyspark.sql import SparkSession

if CLUSTER_TYPE in ['local', 'docker_container']:
    
    spark = SparkSession.builder \
        .master("spark://localhost:7077")\
        .appName("Spark streaming application")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
        .getOrCreate()

elif CLUSTER_TYPE == 'docker_cluster':
    
    # use the provided master
    spark = SparkSession.builder \
        .master("spark://spark-master:7077")\
        .appName("Spark streaming application")\
        .config("spark.executor.memory", "512m")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
        .getOrCreate()

In [ ]:
sc = spark.sparkContext
sc

## Streaming context

The first step of a spark streaming application is the creation of a StreamingContext. The context can be initialized using `StreamingContext(SparkContext, batch_interval`). There could be at most one StreamingContext for each spark application. The processing start when `streamingContext.start()` is called and it can be stopped with `streamingContext.stop()`. If it is stoped without passing `stop(stopSparkContext=False)` the sparkContext is also stopped.

In [ ]:
# create streaming context

For this example spark will read data from a TCP socked. Data are generated by a simple python program that can be found in `utils/producer.py`. The producer will write data on port `5555` of `localhost`. 

The dataset consists of fake credit card transactions.

In [ ]:
# read stream from socket

The first thing we need to to is load the json describing each transaction.

In [ ]:
# load json files

It is possible to print some elements of each batch with `pprint()`. This can be used to explore the RDDs.

In [ ]:
# show elemnts

Start the computations with `ssc.start()` and stop with `ssc.stop(stopSparkContext=False)`. Remember that after this last instruction the streaming context must be defined again.

In [ ]:
# start stream


In [ ]:
# stop stream

Before running the following cells, create again the streaming context and run the cells where the input data source is defined. Remember to skip `pprint` otherwise this operation will be appended to the DAG. 

We may want to convert each batch into a Spark DataFrame to have higher level API. To do that, let's first convert numeric features of the json into python floats and integers. The dictionary can then be used to create a `Row`for each transaction.

In [ ]:
from pyspark.sql import Row

def create_row_rdd(t):
    # set data types and create
    # dataframe rows

row_stream = json_stream.map(create_row_rdd)

The method `DStream.foreachRDD` can be used to apply custom transformations to each batch of data. In our case we are insterested in converting each batch of data into a dataframe and perform operations like finding the number of transactions for each user. Transacion where the user performed more than one transaction, with the flag equal to one,  inside the microbatch and can be flagged as fraudulent.

In [ ]:
from pyspark.sql.functions import concat, col, lit, countDistinct

def process_batch(rdd):
    # convert to df
    
    # find number of transactions for each user when flag = 1 
    
    
    # find suspicious transactions
    
    
    # send allarm
    


row_stream.foreachRDD(process_batch)

In [ ]:
ssc.start()

In [ ]:
# stop streaming context

ssc.stop(stopSparkContext=False)